In [73]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [34]:
df_data = pd.read_csv("train_hongkyu.csv")

df_data.isna().sum()

Survived               0
Pclass                 0
SibSp                  0
Parch                  0
Age_int_scaled       202
Fare_scaled            0
Embarked_C             0
Embarked_Q             0
Embarked_S             0
Cabin_str_A            0
Cabin_str_B            0
Cabin_str_C            0
Cabin_str_D            0
Cabin_str_E            0
Cabin_str_F            0
Cabin_str_G            0
Cabin_str_T            0
Name_title_Master      0
Name_title_Miss        0
Name_title_Mr          0
Name_title_Mrs         0
Sex_female             0
Sex_male               0
dtype: int64

In [35]:
df_data.loc[df_data["Age_int_scaled"].isna(), "Age_int_scaled"] = df_data["Age_int_scaled"].mean()


In [36]:
list_feature = list(df_data.columns)
list_feature.remove('Survived')
X = df_data[list_feature].copy()
y = df_data["Survived"].copy()

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [75]:
para_grid_rf = [
    {'n_estimators': [10, 50, 100, 250, 500, 750, 1000],
     'criterion': ['gini', 'entropy'],
     'max_depth': [1, 2, 3, 4, 5, None],
     'max_features': [1, 3, 5, 7, 9, 11, None]}
]

# clf_rf=RandomForestClassifier(n_estimators=100, max_depth=None)
clf_rf=RandomForestClassifier()
grid_search_rf = GridSearchCV(clf_rf, para_grid_rf)
# clf_rf.fit(X_train,y_train)
grid_search_rf.fit(X_train,y_train)
print(grid_search_rf.best_params_)
print(grid_search_rf.best_score_)

# y_pred_rf=clf_rf.predict(X_test)

# print("Random Forest Accuracy:",metrics.accuracy_score(y_test, y_pred_rf))

{'criterion': 'entropy', 'max_depth': 5, 'max_features': 11, 'n_estimators': 50}
0.8427225806451613


In [79]:
df_result = pd.DataFrame(grid_search_rf.cv_results_)
df_result.sort_values("rank_test_score", inplace=True)

In [80]:
df_result.to_csv('result_rf.csv')

In [81]:
para_grid_xgb = [
    {'n_estimators': [10, 100, 500, 1000],
     'learning_rate': [0.1, 0.005, 0.0001],
     'max_depth': [1, 3, 5, 10],
     'colsample_bytree': [0.1, 0.3, 0.7, 1]}
]

# clf_xgb = XGBClassifier(n_estimators=100, max_depth=4, learning_rate=0.0001)
clf_xgb = XGBClassifier()
grid_search_xgb = GridSearchCV(clf_xgb, para_grid_xgb)
# clf_xgb.fit(X_train, y_train)
grid_search_xgb.fit(X_train, y_train)

# y_pred_xgb = clf_xgb.predict(X_test)
print(grid_search_xgb.best_params_)
print(grid_search_xgb.best_score_)


# print("XGBoost Accuracy:",metrics.accuracy_score(y_test, y_pred_xgb))


{'colsample_bytree': 0.3, 'learning_rate': 0.005, 'max_depth': 10, 'n_estimators': 1000}
0.8443225806451613


In [82]:
df_result_xgb = pd.DataFrame(grid_search_xgb.cv_results_)
df_result_xgb.sort_values("rank_test_score", inplace=True)
df_result_xgb.to_csv("result_xgb.csv")



In [ ]:
## Todo
# 1. Age 결측치 처리를 평균 말고 다르게? (Ex. Na와 Not Na 구분)
# 2. Age int만 남기는 과정 제거? (Master는 실제 어린 남자아이의 호칭)
# 3. Grid search 범위 넓혀서 및 결과 plot